<h1 style = 'text-align: center'> <b>Week 06: Deep Learning For NLP</b> </h1>

- Mentee: Võ Nguyễn Hoàng Kim
- Mentee ID: 240103

In [ ]:
!pip install underthesea
!pip install gensim

import pandas as pd
import numpy as np
import os

import re
import underthesea
from underthesea import word_tokenize
from underthesea import text_normalize
from gensim.models import Word2Vec

import seaborn as sns
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import TfidfVectorizer
from underthesea import text_normalize
from underthesea import word_tokenize
import torch.nn as nn


# <b>1. Load Data path file and Label</b>

In [ ]:
def load_data_path_and_label(base_dir):
    file_paths = []
    labels = []
    for category in os.listdir(base_dir):
        category_dir_path = os.path.join(base_dir, category)
        for text_file in os.listdir(category_dir_path):
            text_file_path = os.path.join(category_dir_path, text_file)
            # append data & correspoding label
            file_paths.append(text_file_path)
            labels.append(category)
    return file_paths, labels

# get path of data txt and the label
base_dir = '/kaggle/input/vietnamese-news-text-classification-corpus/27_Topics/Train/new train'
file_paths, labels = load_data_path_and_label(base_dir)

# <b>2. Encoding Label</b>

In [ ]:
# encoding label
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# <b>3. Preprocess</b>

In [ ]:
def read_text_file(file_path):
    encodings = ['utf-8', 'latin-1', 'windows-1252', 'utf-16']
    for enc in encodings:
        try:
            with open(file_path, 'r', encoding=enc) as file:
                return file.read()
        except (UnicodeDecodeError, FileNotFoundError):
            continue  # Nếu gặp lỗi, thử với mã hóa tiếp theo
    return None  # Trả về None nếu không thể đọc tệp


def preprocess_text(text_file):
    stop_words = read_text_file("/kaggle/input/vietnamese-stop-words/vietnamese-stopwords.txt")
    
    text = text_file.lower()
    # normalize
    normalized_text = text_normalize(text)
    # remove stop words
    stop_words_pattern = r'\b(?:' + '|'.join(map(re.escape, stop_words)) + r')\b'
    cleaned_text = re.sub(stop_words_pattern, ' ', normalized_text)
    cleaned_text = ' '.join(cleaned_text.split())
    tokens = word_tokenize(cleaned_text)
    return tokens

# Load dữ liệu và tiền xử lý
documents = [preprocess_text(read_text_file(file_path)) for file_path in file_paths]

# Tạo mô hình Word2Vec với dữ liệu đã được token hóa
word2vec_model = Word2Vec(sentences=documents, vector_size=100, window=5, min_count=1, workers=4)
word2vec_model.save("word2vec.model")

# Hàm chuyển đổi một văn bản thành vector bằng cách lấy trung bình vector của các từ
def get_average_word2vec(tokens, model, vector_size):
    vectors = []
    for token in tokens:
        if token in model.wv:
            vectors.append(model.wv[token])
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(vector_size)

# <b>4. Create Dataset & DataLoader</b>

In [ ]:
# Cập nhật TextDataset
class TextDataset(Dataset):
    def __init__(self, file_paths, labels, word2vec_model, vector_size=100):
        self.file_paths = file_paths
        self.labels = torch.tensor(labels, dtype=torch.long)
        self.word2vec_model = word2vec_model
        self.vector_size = vector_size

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, index):
        file_path = self.file_paths[index]
        text = read_text_file(file_path)
        tokens = preprocess_text(text)
        
        # Chuyển đổi văn bản thành vector
        word2vec_vector = get_average_word2vec(tokens, self.word2vec_model, self.vector_size)
        
        input_ids = torch.tensor(word2vec_vector, dtype=torch.float32)  # Chuyển thành tensor
        label = self.labels[index]

        return {
            "input_ids": input_ids,
            "label": label
        }

# Khởi tạo dataset và dataloader
dataset = TextDataset(file_paths, encoded_labels, word2vec_model)
train_dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# <b>5. Create Model</b>

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, dropout=0.5):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.unsqueeze(1)  # Thêm một chiều cho batch
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])  # Lấy đầu ra cuối cùng từ LSTM
        return out

def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()  # Đặt mô hình ở chế độ huấn luyện
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        running_loss = 0.0
        for batch in train_loader:
            optimizer.zero_grad()  # Đặt lại gradient

            # Chuyển dữ liệu và nhãn sang GPU (nếu có)
            input_ids = batch['input_ids'].to(device)
            labels = batch['label'].to(device)

            # Forward pass
            outputs = model(input_ids)  # Dự đoán đầu ra từ mô hình
            loss = criterion(outputs, labels)  # Tính toán loss

            # Backward pass và tối ưu hóa
            loss.backward()  # Tính gradient
            optimizer.step()  # Cập nhật trọng số

            running_loss += loss.item()  # Tính tổng loss
        print(f'Loss: {running_loss/len(train_loader):.4f}')  # In ra loss trung bình của mỗi epoch



# <b>6. Train Model</b>

In [ ]:
# Khởi tạo mô hình với kích thước đầu vào phù hợp với Word2Vec
input_size = 100  # Kích thước vector của Word2Vec
hidden_size = 128
output_size = len(label_encoder.classes_)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Khởi tạo mô hình
model = LSTMClassifier(input_size, hidden_size, output_size)

# Chuyển mô hình sang GPU nếu có
model = model.to(device)

# Sử dụng CrossEntropyLoss cho bài toán phân loại
criterion = nn.CrossEntropyLoss()

# Sử dụng Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)  # lr là learning rate

# Huấn luyện
train_model(model, train_dataloader, criterion, optimizer, epochs=10)

# Lưu trọng số của mô hình
torch.save(model.state_dict(), '/kaggle/working/LSTM_model_word2vec_weights.pth')


# <b>7. Test Model</b>

## <b>a. Create Test data</b>

In [ ]:
test_path_dir = '/kaggle/input/vietnamese-news-text-classification-corpus/27_Topics/Test/new test'
test_file_paths, test_labels = load_data_path_and_label(test_path_dir)
test_documents = [preprocess_text(read_text_file(file_path)) for file_path in test_file_paths]

# Khởi tạo dataset và dataloader cho dữ liệu kiểm thử
test_dataset = TextDataset(test_file_paths, test_labels, word2vec_model, test_documents)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# <b>b. Test model</b>

In [ ]:
def test_model(model, test_dataloader, criterion):
    model.eval()  # Chuyển mô hình sang chế độ đánh giá
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():  # Không tính gradient khi kiểm thử
        for batch in test_dataloader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['label'].to(device)

            # Forward pass
            outputs = model(input_ids)
            loss = criterion(outputs, labels)

            total_loss += loss.item()

            # Lấy nhãn dự đoán
            _, predicted_labels = torch.max(outputs, dim=1)
            correct_predictions += (predicted_labels == labels).sum().item()
            total_samples += labels.size(0)

    avg_loss = total_loss / len(test_dataloader)
    accuracy = correct_predictions / total_samples

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.4f}")
    return avg_loss, accuracy

# Kiểm thử mô hình trên tập dữ liệu kiểm thử
test_loss, test_accuracy = test_model(model, test_dataloader, criterion)
